# Corrections

In [1]:
import pandas as pd

In [7]:
df = pd.read_csv('data.csv')
df.head(1)

,text,author,tags,born_date,born_location,image_urls,images
0,"“The person, be it gentleman or lady, who has ...",Jane Austen,"aliteracy,books,classic,humor","December 16, 1775","in Steventon Rectory, Hampshire, The United Ki...",https://upload.wikimedia.org/wikipedia/commons...,[{'url': 'https://upload.wikimedia.org/wikiped...


In [20]:
df = pd.read_csv('data.csv')

df['text'] = df['text'].str.lower()

replacements = ['“','”', ',', '.' ]
for rep in replacements:
    df['text'] = df['text'].str.replace(rep, '')
    
    
#df['n_words'] = df['text'].str.split().str.len()    
df['n_words'] = df['text'].str.split().apply(len)

df.loc[0:5, 'text']

/Users/blanchardnicolas/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  import sys


0    the person be it gentleman or lady who has not...
1        a day without sunshine is like you know night
2    a woman is like a tea bag; you never know how ...
3    life is what happens to us while we are making...
4    i may not have gone where i intended to go but...
5    i have not failed i've just found 10000 ways t...
Name: text, dtype: object

In [14]:
df

,text,author,tags,born_date,born_location,image_urls,images,n_words
0,the person be it gentleman or lady who has not...,Jane Austen,"aliteracy,books,classic,humor","December 16, 1775","in Steventon Rectory, Hampshire, The United Ki...",https://upload.wikimedia.org/wikipedia/commons...,[{'url': 'https://upload.wikimedia.org/wikiped...,<bound method StringMethods.len of <pandas.cor...
1,a day without sunshine is like you know night,Steve Martin,"humor,obvious,simile","August 14, 1945","in Waco, Texas, The United States",https://upload.wikimedia.org/wikipedia/commons...,[{'url': 'https://upload.wikimedia.org/wikiped...,<bound method StringMethods.len of <pandas.cor...
2,a woman is like a tea bag; you never know how ...,Eleanor Roosevelt,misattributed-eleanor-roosevelt,"October 11, 1884",in The United States,https://upload.wikimedia.org/wikipedia/commons...,[{'url': 'https://upload.wikimedia.org/wikiped...,<bound method StringMethods.len of <pandas.cor...
3,life is what happens to us while we are making...,Allen Saunders,"fate,life,misattributed-john-lennon,planning,p...","April 24, 1899",in The United States,https://upload.wikimedia.org/wikipedia/en/thum...,[{'url': 'https://upload.wikimedia.org/wikiped...,<bound method StringMethods.len of <pandas.cor...
4,i may not have gone where i intended to go but...,Douglas Adams,"life,navigation","March 11, 1952","in Cambridge, England, The United Kingdom",https://upload.wikimedia.org/wikipedia/commons...,[{'url': 'https://upload.wikimedia.org/wikiped...,<bound method StringMethods.len of <pandas.cor...
...,...,...,...,...,...,...,...,...
95,"the truth"" dumbledore sighed ""it is a beautifu...",J.K. Rowling,truth,"July 31, 1965","in Yate, South Gloucestershire, England, The U...",https://upload.wikimedia.org/wikipedia/commons...,[{'url': 'https://upload.wikimedia.org/wikiped...,<bound method StringMethods.len of <pandas.cor...
96,you never really understand a person until you...,Harper Lee,better-life-empathy,"April 28, 1926","in Monroeville, Alabama, The United States",https://upload.wikimedia.org/wikipedia/commons...,[{'url': 'https://upload.wikimedia.org/wikiped...,<bound method StringMethods.len of <pandas.cor...
97,to die will be an awfully big adventure,J.M. Barrie,"adventure,love","May 09, 1860","in Kirriemuir, Angus, Scotland, The United Kin...",https://upload.wikimedia.org/wikipedia/commons...,[{'url': 'https://upload.wikimedia.org/wikiped...,<bound method StringMethods.len of <pandas.cor...
98,it takes courage to grow up and become who you...,E.E. Cummings,courage,"October 14, 1894","in Cambridge, Massachusetts, The United States",https://upload.wikimedia.org/wikipedia/commons...,[{'url': 'https://upload.wikimedia.org/wikiped...,<bound method StringMethods.len of <pandas.cor...
